In [1]:
! pip install transformers datasets

INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 18.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 13.1 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.


In [2]:
from google.colab import files
uploded=files.upload()

#dataset["train"][100]

Saving labeled_data_freemarket_final.conll to labeled_data_freemarket_final.conll


In [3]:
!pip install datasets

In [6]:
from datasets import load_dataset

# Load the dataset
dataset = load_dataset('text', data_files='labeled_data_freemarket_final.conll', split='train')

# Access the 100th example
example = dataset[100]
print(example)            # Print the entire example
print(example["text"])    # Print just the text

Generating train split: 0 examples [00:00, ? examples/s]

{'text': 'ውስጥ O'}
ውስጥ O


In [7]:
import pandas as pd

# Initialize examples list to hold token-label pairs
examples = []

# Open the .conll file and read it line by line
with open('labeled_data_freemarket_final.conll', 'r', encoding='utf-8') as file:
    current_example = []
    for line in file:
        line = line.strip()
        if line:  # If the line is not empty
            token, label = line.split()
            current_example.append((token, label))
        else:  # Empty line indicates end of an example
            if current_example:
                examples.append(current_example)
                current_example = []

    # Add the last example if there's no trailing newline
    if current_example:
        examples.append(current_example)

# Example output: Print the first example
print(examples[0])

# Flatten the examples into a DataFrame
data = []
for example in examples:
    for token, label in example:
        data.append((token, label))

df = pd.DataFrame(data, columns=['Token', 'Label'])

# Print the first few rows of the DataFrame
print(df.head())

[('ለተለያዪ', 'O'), ('አትክልቶች', 'O'), ('ስላይስ', 'O'), ('ለማድረግ', 'O'), ('የሚረዳ', 'O'), ('ለማጠብ', 'O'), ('በጣም', 'O'), ('ቀላል', 'O'), ('ዉሃዉን', 'O'), ('የሚያጠል', 'O'), ('ልዩ', 'O'), ('ዕቃ', 'O'), ('አምስት', 'O'), ('የተለያየ', 'O'), ('ቅርፅ', 'O'), ('መቁረጫ', 'O'), ('ያለው', 'O'), ('ዋጋ፦', 'B-PRICE'), ('900', 'I-PRICE'), ('ብር', 'I-PRICE'), ('አድራሻ', 'O'), ('1⃣ፒያሳ', 'I-LOC'), ('ጣይቱ', 'I-LOC'), ('ሆቴል', 'I-LOC'), ('ጊቢ', 'O'), ('ውስጥ', 'O'), ('ቢሮ', 'O'), ('ቁ04', 'O'), ('2⃣መገናኛ', 'I-LOC'), ('መተባበር', 'I-LOC'), ('ሕንፃ', 'O'), ('3ኛ', 'O'), ('ፎቅ', 'O'), ('ቢሮ', 'O'), ('ቁ316', 'O'), ('3⃣ሳር', 'I-LOC'), ('ቤት', 'O'), ('ካናዳ', 'I-LOC'), ('ኤምባሲ', 'I-LOC'), ('ፊትለፊት', 'O'), ('ሸዋ', 'I-LOC'), ('ሱፐርማርኬት', 'I-LOC'), ('ያለበት', 'O'), ('ህንፃ', 'O'), ('2ኛ', 'O'), ('ፎቅ', 'O'), ('ቢሮ', 'O'), ('ቁ11', 'O'), ('4⃣', 'O'), ('ጀሞ', 'I-LOC'), ('መስታወት', 'O'), ('ፍብሪካ', 'O'), ('ፊት', 'O'), ('ለፊት', 'O'), ('ከፍደም', 'I-LOC'), ('ሞል', 'O'), ('ግራውድ', 'I-LOC'), ('ላይ', 'O'), ('የሱቅ', 'O'), ('ቁጥር', 'O'), ('39', 'O'), ('ለማዘዝ', 'O'), ('እነዚን', 'O'), ('ቁጥሮች', 'O'), ('ይጠቀሙ', 

In [9]:
from transformers import AutoTokenizer
from datasets import load_dataset

# Load the dataset as plain text
dataset = load_dataset('text', data_files='labeled_data_freemarket_final.conll', split='train')

# Load the BERT tokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

# Function to process each example and tokenize the text
def tokenize_function(examples):
    tokens = [line.split()[0] for line in examples["text"] if line.strip()]

    # Tokenize the tokens with padding and truncation to max 512 tokens
    tokenized_output = tokenizer(tokens, is_split_into_words=True, padding="max_length", truncation=True, max_length=512)

    return tokenized_output

# To avoid Arrow serialization limitations, you can disable the use of arrow format in map
tokenized_datasets = dataset.map(tokenize_function, batched=True, remove_columns=["text"], num_proc=4, load_from_cache_file=False)

# Preview the tokenized output
print(tokenized_datasets[0])

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Map (num_proc=4):   0%|          | 0/210454 [00:00<?, ? examples/s]

{'input_ids': 101, 'token_type_ids': 0, 'attention_mask': 1}


In [11]:
from transformers import AutoTokenizer
from datasets import load_dataset

# Load the dataset as plain text again
dataset = load_dataset('text', data_files='labeled_data_freemarket_final.conll', split='train')

# Load the BERT tokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

# Function to process each example and tokenize the text
def tokenize_function(examples):
    tokens = [line.split()[0] for line in examples["text"] if line.strip()]

    # Tokenize the tokens with padding and truncation to max 512 tokens
    tokenized_output = tokenizer(tokens, is_split_into_words=True, padding="max_length", truncation=True, max_length=512)

    return tokenized_output

# Apply the tokenizer to the dataset using map
tokenized_datasets = dataset.map(tokenize_function, batched=True, remove_columns=["text"])

# Now split the tokenized dataset into train and test datasets manually if needed
train_size = int(0.8 * len(tokenized_datasets))  # 80% for training
test_size = len(tokenized_datasets) - train_size  # Remaining for testing

# Create the train and test datasets
small_train_dataset = tokenized_datasets.select(range(train_size)).shuffle(seed=42)
small_eval_dataset = tokenized_datasets.select(range(train_size, len(tokenized_datasets))).shuffle(seed=42)

# Preview the small datasets
print(small_train_dataset[0])
print(small_eval_dataset[0])

Map:   0%|          | 0/210454 [00:00<?, ? examples/s]

{'input_ids': 1604, 'token_type_ids': 0, 'attention_mask': 1}
{'input_ids': 1604, 'token_type_ids': 0, 'attention_mask': 1}


In [12]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=5)

model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [13]:
from transformers import TrainingArguments

training_args = TrainingArguments(output_dir="test_trainer")

In [14]:
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 7.3 MB/s eta 0:00:00


In [ ]:
import numpy as np
import evaluate
from transformers import Trainer, TrainingArguments, AutoTokenizer, AutoModelForTokenClassification
from datasets import load_dataset

# Load the dataset
dataset = load_dataset('text', data_files='labeled_data_freemarket_final.conll', split='train')

# Extract unique labels and create a mapping
unique_labels = set()
for entry in dataset['text']:
    line = entry.strip()
    if line:  # If the line is not empty
        token, label = line.split()
        unique_labels.add(label)

# Create a label to ID mapping
label_to_id = {label: idx for idx, label in enumerate(unique_labels)}
id_to_label = {idx: label for label, idx in label_to_id.items()}

# Prepare the labels in the dataset
# This is an example; modify as needed to fit your dataset structure.
dataset = dataset.map(lambda x: {"labels": [label_to_id[label] for label in x["text"].split()[1:]]})

# Load the tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")
model = AutoModelForTokenClassification.from_pretrained("bert-base-cased", num_labels=len(unique_labels))

# Updated tokenization function
def tokenize_function(examples):
    tokenized_input = tokenizer(
        examples["text"],
        truncation=True,
        padding="max_length",
        max_length=512,
        is_split_into_words=False
    )

    labels = []

    for i in range(len(examples["text"])):
        label_ids = [-100] * len(tokenized_input["input_ids"][i])  # Initialize to -100
        # Get the original labels for the current example
        original_labels = examples["labels"][i]

        # Set the appropriate labels for each token
        for j in range(len(original_labels)):
            if j < len(label_ids):  # Only assign if within the range
                label_ids[j] = original_labels[j]  # Use the label for the token

        labels.append(label_ids)

    tokenized_input["labels"] = labels
    return tokenized_input

# Tokenize the dataset
tokenized_datasets = dataset.map(tokenize_function, batched=True, remove_columns=["text"])

# Manual train/test split
train_size = int(0.8 * len(tokenized_datasets))
small_train_dataset = tokenized_datasets.select(range(train_size))
small_eval_dataset = tokenized_datasets.select(range(train_size, len(tokenized_datasets)))

# Load evaluation metric
metric = evaluate.load("accuracy")

# Compute metrics function
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions.flatten(), references=labels.flatten())

# Trainer arguments
training_args = TrainingArguments(
    output_dir="test_trainer",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=1,
)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    compute_metrics=compute_metrics,
)

# Train the model
trainer.train()

Map:   0%|          | 0/210454 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/210454 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss


Epoch,Training Loss,Validation Loss
